In [1]:
import nest_asyncio
import asyncio

# Apply nest_asyncio BEFORE importing qnexus to patch asyncio properly
nest_asyncio.apply()

import qnexus as qnx
import pytket as pk
import numpy as np
import scipy as sc
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from pytket.utils import QubitPauliOperator
from pytket.passes import DecomposeBoxes
import numpy as np
from pytket.circuit import Unitary2qBox, Unitary1qBox
import operators.plaqOps as plaq_states
from qiskit import QuantumCircuit, transpile
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import (
        DecomposeBoxes,
        FullPeepholeOptimise,
        CliffordSimp,
        RemoveRedundancies,
        SimplifyInitial
    )
from pytket.circuit import Qubit, Bit
import datetime

In [2]:
# Login to qnexus to access Quantinuum credentials
qnx.login()

project = qnx.projects.get_or_create(name="U1_TEBD")
qnx.context.set_active_project(project)
config = qnx.QuantinuumConfig(device_name="H2-Emulator")

KeyboardInterrupt: 

In [3]:
l = 0.5
states = int(2*l+1)
qubits = int(np.ceil(np.log2(states)))

In [4]:
def get_local_circuit(g2, tau):
    oL, Lsq, ImTrP = plaq_states._getPlaqStateOps(l, g2)
    pad = 2**qubits-states
    oL, Lsq, ImTrP = np.pad(oL, (0,pad)), np.pad(Lsq, (0,pad)), np.pad(ImTrP, (0,pad))
    operator_matrix = sc.linalg.expm(-1j*tau*np.matrix(g2/2 * 4* Lsq + 1/2/g2*ImTrP))

    circuit = Circuit(qubits)

    
    if l<1:
        unitary_box = Unitary1qBox(operator_matrix)
        circuit.add_unitary1qbox(unitary_box, 0)
        DecomposeBoxes().apply(circuit)
    elif l<2:
        unitary_box = Unitary2qBox(operator_matrix)
        circuit.add_unitary2qbox(unitary_box, 0, 1)
        DecomposeBoxes().apply(circuit)
    else:
        print("Unsupported")
    return circuit
    

In [5]:
def get_interaction(g2, tau, approximation_degree=1.0): #check prefactors
    oL, Lsq, ImTrP = plaq_states._getPlaqStateOps(l, g2)
    pad = 2**qubits-states
    oL, Lsq, ImTrP = np.pad(oL, (0,pad)), np.pad(Lsq, (0,pad)), np.pad(ImTrP, (0,pad))

    operator_matrix = sc.linalg.expm(-1j*np.matrix(g2/2* tau * np.kron(oL, oL)))

    qc = QuantumCircuit(2*qubits)
    qc.unitary(operator_matrix, list(np.arange(2*qubits)), label='U4')
    # approximation_degree: 1.0 = exact, 0.0 = most approximate (reduces gates)
    qc_decomposed = transpile(qc, basis_gates=['u3', 'cx'], optimization_level=3,
                              approximation_degree=approximation_degree)
    circuit = qiskit_to_tk(qc_decomposed)

    DecomposeBoxes().apply(circuit)

    #print(f"Number of gates: {circuit.n_gates}")
    #print(f"Depth: {circuit.depth()}")
    #print(f"Two-qubit gate count: {circuit.n_gates_of_type(pk.circuit.OpType.CX)}")

    ## Check unitary fidelity (verify decomposition is accurate)
    #original_unitary = np.array(operator_matrix)
    #reconstructed_unitary = circuit.get_unitary()

    ## Calculate process fidelity: F = |tr(U† V)|² / d²
    #d = original_unitary.shape[0]
    #trace_product = np.trace(original_unitary.conj().T @ reconstructed_unitary)
    #fidelity = np.abs(trace_product)**2 / d**2

    #print(f"\nFidelity: {fidelity:.10f}")
    #print(f"Infidelity: {1 - fidelity:.2e}")

    return circuit

In [6]:
# Test approximation_degree effect on gate count and fidelity
print("="*70)
print("APPROXIMATION DEGREE vs GATE COUNT & FIDELITY")
print("="*70)

oL, Lsq, ImTrP = plaq_states._getPlaqStateOps(1, 1)
oL, Lsq, ImTrP = np.pad(oL, (0,1)), np.pad(Lsq, (0,1)), np.pad(ImTrP, (0,1))
operator_matrix = sc.linalg.expm(-1j*np.matrix(1/2 * 0.05 * np.kron(oL, oL)))
original_unitary = np.array(operator_matrix)

results = []
for app_deg in [1.0, 0.999, 0.998, 0.997, 0.996, 0.995, 0.99, 0.985, 0.98]:
    try:
        circuit = get_interaction(1, 0.05, approximation_degree=app_deg)
        
        try:
            reconstructed_unitary = circuit.get_unitary()
            d = original_unitary.shape[0]
            trace_product = np.trace(original_unitary.conj().T @ reconstructed_unitary)
            fidelity = np.abs(trace_product)**2 / d**2
        except:
            fidelity = 1.0
        
        cx_count = circuit.n_gates_of_type(pk.circuit.OpType.CX)
        total_gates = circuit.n_gates
        
        results.append({
            'app_deg': app_deg,
            'gates': total_gates,
            'cx': cx_count,
            'fidelity': fidelity
        })
        
        print(f"Approx {app_deg:.1f}: {total_gates:3d} gates (CX: {cx_count:2d}), Fidelity: {fidelity:.10f}")
    except Exception as e:
        print(f"Approx {app_deg:.1f}: ERROR - {e}")

APPROXIMATION DEGREE vs GATE COUNT & FIDELITY
Approx 1.0:   8 gates (CX:  2), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000
Approx 1.0:   0 gates (CX:  0), Fidelity: 1.0000000000


In [7]:
def grid(x,y, nx, ny):
    assert(x<nx)
    assert(y<ny)
    ind1 = nx*y+x
    return np.arange(qubits)+ind1*qubits

def get_index_set(nx, ny):
    indices_inter = []
    indices_local = []
    for i in range(nx):
        for j in range(ny):
            indices_local.append(grid(i,j, nx, ny))
            if j<ny-1:
                indices_inter.append(np.array([grid(i,j, nx, ny),grid(i,j+1, nx, ny)]))
            if i<nx-1:
                indices_inter.append(np.array([grid(i,j, nx, ny),grid(i+1,j, nx, ny)]))
    return np.array(indices_inter), np.array(indices_local)

In [8]:
def get_trotter_step(nx, ny, g2, tau):
    trotter_step = Circuit(nx*ny*qubits)
    local_gate = get_local_circuit(g2, 0.5*tau)
    interact_gate = get_interaction(g2, 1*tau, approximation_degree=1)

    index_set_interact, index_set_local = get_index_set(nx, ny)
    for link in index_set_interact:
        qs1 = link[0]
        qs2 = link[1]
        trotter_step.add_circuit(interact_gate, list(np.append(qs1,qs2)))
    for site in index_set_local:
        qs1 = site
        trotter_step.add_circuit(local_gate, list(qs1))
    for link in index_set_interact[::-1]:
        qs1 = link[0]
        qs2 = link[1]
        trotter_step.add_circuit(interact_gate, list(np.append(qs1,qs2)))

    return trotter_step

In [9]:
def optimize_circuit(circuit):
    FullPeepholeOptimise().apply(circuit)
    CliffordSimp().apply(circuit)
    RemoveRedundancies().apply(circuit)
    return circuit

def profile_circuit(circuit):
    print("=" * 50)
    print("CIRCUIT PROFILE")
    print("=" * 50)
    print(f"Qubit count:        {circuit.n_qubits}")
    print(f"Total gates:        {circuit.n_gates}")
    print(f"Circuit depth:      {circuit.depth()}")
    print(f"2-qubit gates (CX): {circuit.n_gates_of_type(pk.circuit.OpType.CX)}")
    print(f"Single-qubit gates: {circuit.n_gates_of_type(pk.circuit.OpType.U3) + circuit.n_gates_of_type(pk.circuit.OpType.U1) + circuit.n_gates_of_type(pk.circuit.OpType.U2)}")
    print(f"Gate density:       {circuit.n_gates / (circuit.depth() + 1):.2f} gates/layer")
    print("=" * 50)

def compile_for_h2(circuit):
    """Compile circuit for Quantinuum H2-1 device"""
    from pytket.extensions.quantinuum import QuantinuumBackend
    
    # Create a copy to avoid modifying original
    compiled = circuit.copy()
    
    # Apply general optimization
    optimize_circuit(compiled)
    
    # Get the backend for H2-1
    backend = QuantinuumBackend(device_name="H2-1")
    
    # Compile to the device
    compiled = backend.get_compiled_circuit(compiled)
    
    print("\n✓ Circuit compiled for Quantinuum H2-1")
    print(f"Required qubits: {compiled.n_qubits}")
    print(f"Native gate depth: {compiled.depth()}")
    
    return compiled

async def simulate_circuit_fidelity_async(circuit, shots=1000, disable_noise=False):
    # Get ideal unitary
    ideal_unitary = circuit.get_unitary()
    n_qubits = circuit.n_qubits
    
    # Create a copy and add measurements
    compiled = circuit.copy()
    optimize_circuit(compiled)
    
    # Add classical bits and measurement gates
    compiled.add_c_register("m", n_qubits)
    for i in range(n_qubits):
        args_list = [Qubit(i), Bit("m", i)]
        compiled.add_gate(pk.circuit.OpType.Measure, args_list)
    
    print("=" * 50)
    print("H2 EMULATOR SIMULATION (via qnexus)")
    print("=" * 50)
    print(f"Ideal dimension:    {ideal_unitary.shape[0]}×{ideal_unitary.shape[0]}")
    print(f"Shots:              {shots}")
    print(f"Noise model:        {'Disabled' if disable_noise else 'Enabled'}")
    
    # Step 1: Upload circuit to qnexus
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
    ref_circuit = qnx.circuits.upload(circuit=compiled, name=f"circuit-{timestamp}")
    print(f"✓ Circuit uploaded")
    
    # Step 2: Compile circuit
    # Pass noisy_simulation parameter during config creation (config is frozen, cannot modify after)
    config = qnx.QuantinuumConfig(device_name="H2-Emulator", noisy_simulation=not disable_noise)
    
    ref_compile_job = qnx.start_compile_job(
        programs=[ref_circuit],
        backend_config=config,
        optimisation_level=2,
        name=f"compile-job-{timestamp}"
    )
    
    # Use asyncio.sleep to yield control while waiting
    import asyncio as aio
    while True:
        try:
            result = qnx.jobs.results(ref_compile_job)
            if result:
                break
        except:
            pass
        await aio.sleep(1)
    
    ref_compiled = result[0].get_output()
    print(f"✓ Circuit compiled")
    
    # Step 3: Execute on H2-Emulator
    ref_execute_job = qnx.start_execute_job(
        programs=[ref_compiled],
        n_shots=[shots],
        backend_config=config,
        name=f"execute-job-{timestamp}"
    )
    
    while True:
        try:
            result = qnx.jobs.results(ref_execute_job)
            if result:
                break
        except:
            pass
        await aio.sleep(1)
    
    print(f"✓ Execution complete")
    
    # Step 4: Get results
    ref_result = result[0]
    backend_result = ref_result.download_result()
    
    counts = {}
    if hasattr(backend_result, "get_counts"):
        try:
            counts = dict(backend_result.get_counts())
        except Exception:
            counts = {}
    
    if not counts:
        distribution = backend_result.get_empirical_distribution()
        
        def _distribution_to_dict(dist):
            if hasattr(dist, "as_dict"):
                return dist.as_dict()
            if hasattr(dist, "to_dict"):
                return dist.to_dict()
            if hasattr(dist, "dict"):
                try:
                    return dist.dict()
                except TypeError:
                    return dist.dict
            if hasattr(dist, "outcomes") and hasattr(dist, "probabilities"):
                return {outcome: prob for outcome, prob in zip(dist.outcomes, dist.probabilities)}
            if hasattr(dist, "samples"):
                try:
                    return dict(dist.samples)
                except Exception:
                    return {}
            try:
                return dict(dist)
            except Exception:
                return {}
        
        dist_dict = _distribution_to_dict(distribution)
        counts = {state: int(round(prob * shots)) for state, prob in dist_dict.items()}
    
    print(f"Measurement counts: {counts}")
    
    # For ideal circuits, dominant state should have high probability
    if counts:
        max_prob = max(counts.values()) / shots
        print(f"Max probability:    {max_prob:.4f}")
    
    print("=" * 50)
    
    return counts

def simulate_circuit_fidelity(circuit, shots=1000, disable_noise=False):
    """Simulate circuit using H2 Emulator via qnexus
    
    Parameters:
    -----------
    circuit : pytket.Circuit
        The circuit to simulate
    shots : int
        Number of measurement shots (default: 1000)
    disable_noise : bool
        If True, disable noise model in H2 Emulator (default: False)
    """
    try:
        return asyncio.run(simulate_circuit_fidelity_async(circuit, shots, disable_noise))
    except Exception as e:
        print(f"H2 Emulator error: {e}")
        print("Note: Ensure qnx.login() has been called to authenticate")
        import traceback
        traceback.print_exc()
        return None


In [10]:
circuit = get_trotter_step(4, 4, 1, 0.05)
print("BEFORE H2-1 compilation:")
profile_circuit(circuit)

print("\nCompiling for H2-1...")
h2_circuit = compile_for_h2(circuit)

print("\nAFTER H2-1 compilation:")
profile_circuit(h2_circuit)

BEFORE H2-1 compilation:
CIRCUIT PROFILE
Qubit count:        16
Total gates:        400
Circuit depth:      121
2-qubit gates (CX): 96
Single-qubit gates: 288
Gate density:       3.28 gates/layer

Compiling for H2-1...

✓ Circuit compiled for Quantinuum H2-1
Required qubits: 16
Native gate depth: 36

AFTER H2-1 compilation:
CIRCUIT PROFILE
Qubit count:        16
Total gates:        133
Circuit depth:      36
2-qubit gates (CX): 0
Single-qubit gates: 0
Gate density:       3.59 gates/layer


In [ ]:
# Test simulation and fidelity
circuit = get_trotter_step(2, 1, 1, 0.05)
#simulate_circuit_fidelity(circuit, disable_noise=True)